In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
import re
import string
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import RandomizedSearchCV

# Загрузка данных
data = pd.read_csv('fake_or_real_news.csv')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Разделение данных на признаки (текст) и метки (label)
X = data['text']
y = data['label']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация текста с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Модель логистической регрессии
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)

# Предсказание и оценка модели
y_pred_lr = lr_model.predict(X_test_tfidf)
print("Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Logistic Regression Accuracy:  0.9155485398579322
              precision    recall  f1-score   support

        FAKE       0.90      0.93      0.92       628
        REAL       0.93      0.90      0.91       639

    accuracy                           0.92      1267
   macro avg       0.92      0.92      0.92      1267
weighted avg       0.92      0.92      0.92      1267



In [ ]:
# Модель наивного Байеса
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Предсказание и оценка модели
y_pred_nb = nb_model.predict(X_test_tfidf)
print("Naive Bayes Accuracy: ", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))


Naive Bayes Accuracy:  0.8453038674033149
              precision    recall  f1-score   support

        FAKE       0.98      0.71      0.82       628
        REAL       0.77      0.98      0.87       639

    accuracy                           0.85      1267
   macro avg       0.87      0.84      0.84      1267
weighted avg       0.87      0.85      0.84      1267



In [ ]:
# Преобразование меток в числовой формат
data['label'] = data['label'].map({'FAKE': 0, 'REAL': 1})

# Разделение данных на признаки (текст) и метки (label)
X = data['text']
y = data['label']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация текста с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Преобразование данных в формат DMatrix для XGBoost
dtrain = xgb.DMatrix(X_train_tfidf, label=y_train)
dtest = xgb.DMatrix(X_test_tfidf, label=y_test)

# Параметры модели
params = {
    'objective': 'binary:logistic',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Обучение модели
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# Предсказание и оценка модели
y_pred_xgb = (xgb_model.predict(dtest) > 0.5).astype(int)
print("XGBoost Accuracy: ", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:52:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy:  0.9123914759273876
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       628
           1       0.92      0.91      0.91       639

    accuracy                           0.91      1267
   macro avg       0.91      0.91      0.91      1267
weighted avg       0.91      0.91      0.91      1267



In [ ]:
print("XGBoost Accuracy: ", accuracy_score(y_test, y_pred_xgb))

XGBoost Accuracy:  0.9123914759273876


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

# Определение сетки гиперпараметров для логистической регрессии
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Определение модели с Grid Search и кросс-валидацией
grid_search_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=StratifiedKFold(5), scoring='accuracy')

# Обучение модели
grid_search_lr.fit(X_train_tfidf, y_train)

# Лучшая комбинация гиперпараметров
print("Best Hyperparameters for Logistic Regression: ", grid_search_lr.best_params_)

# Оценка производительности на тестовом наборе
best_lr = grid_search_lr.best_estimator_
y_pred_lr_optimized = best_lr.predict(X_test_tfidf)
print("Optimized Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred_lr_optimized))
print(classification_report(y_test, y_pred_lr_optimized))


Best Hyperparameters for Logistic Regression:  {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Optimized Logistic Regression Accuracy:  0.936069455406472
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       628
           1       0.94      0.93      0.94       639

    accuracy                           0.94      1267
   macro avg       0.94      0.94      0.94      1267
weighted avg       0.94      0.94      0.94      1267

